In [ ]:
#test.py
# -*- coding: utf-8 -*-
import math
import sys
import pygame
from pygame.locals import *
# import pygame.mixer # サウンドモジュールは使わないのでコメントアウト

# 画面サイズ
SCREEN = Rect(0, 0, 600, 600)

# 画像ファイルのパス
PADDLE_IMG_PATH = "image/paddle.png"
BLOCK_IMG_PATH = "image/backsnow.png"
BALL_IMG_PATH = "image/ball.png"

# バドルのスプライトクラス
class Paddle(pygame.sprite.Sprite):
    # コンストラクタ（初期化メソッド）
    def __init__(self, filename):
        pygame.sprite.Sprite.__init__(self, self.containers)
        self.image = pygame.image.load(filename).convert()
        self.rect = self.image.get_rect()
        self.rect.bottom = SCREEN.bottom - 20      # パドルのy座標

    def update(self):
        self.rect.centerx = pygame.mouse.get_pos()[0]  # マウスのx座標をパドルのx座標に
        self.rect.clamp_ip(SCREEN)                      # ゲーム画面内のみで移動

# ボールのスプライトクラス
class Ball(pygame.sprite.Sprite):
    # コンストラクタ（初期化メソッド）
    def __init__(self, filename, paddle, blocks, score, speed, angle_left, angle_right):
        pygame.sprite.Sprite.__init__(self, self.containers)
        self.image = pygame.image.load(filename).convert()
        self.rect = self.image.get_rect()
        self.dx = self.dy = 0  # ボールの速度
        self.paddle = paddle  # パドルへの参照
        self.blocks = blocks  # ブロックグループへの参照
        self.update = self.start # ゲーム開始状態に更新
        self.score = score
        self.hit = 0  # 連続でブロックを壊した回数
        self.speed = speed # ボールの初期速度
        self.angle_left = angle_left # パドルの反射方向(左端:135度）
        self.angle_right = angle_right # パドルの反射方向(右端:45度）

    # ゲーム開始状態（マウスを左クリック時するとボール射出）
    def start(self):
        # ボールの初期位置(パドルの上)
        self.rect.centerx = self.paddle.rect.centerx
        self.rect.bottom = self.paddle.rect.top

        # 左クリックでボール射出
        if pygame.mouse.get_pressed()[0] == 1:
            self.dx = 0
            self.dy = -self.speed
            self.update = self.move

    # ボールの挙動
    def move(self):
        self.rect.centerx += self.dx
        self.rect.centery += self.dy

        # 壁との反射
        if self.rect.left < SCREEN.left:    # 左側
            self.rect.left = SCREEN.left
            self.dx = -self.dx              # 速度を反転
        if self.rect.right > SCREEN.right:  # 右側
            self.rect.right = SCREEN.right
            self.dx = -self.dx
        if self.rect.top < SCREEN.top:      # 上側
            self.rect.top = SCREEN.top
            self.dy = -self.dy

        # パドルとの反射(左端:135度方向, 右端:45度方向, それ以外:線形補間)
        # 2つのspriteが接触しているかどうかの判定
        if self.rect.colliderect(self.paddle.rect) and self.dy > 0:
            self.hit = 0                                    # 連続ヒットを0に戻す
            (x1, y1) = (self.paddle.rect.left - self.rect.width, self.angle_left)
            (x2, y2) = (self.paddle.rect.right, self.angle_right)
            x = self.rect.left                              # ボールが当たった位置
            y = (float(y2-y1)/(x2-x1)) * (x - x1) + y1  # 線形補間
            angle = math.radians(y)                         # 反射角度
            self.dx = self.speed * math.cos(angle)
            self.dy = -self.speed * math.sin(angle)
            # self.paddle_sound.play()                      # 反射音をコメントアウト

        # ボールを落とした場合
        if self.rect.top > SCREEN.bottom:
            self.update = self.start                        # ボールを初期状態に
            # self.gameover_sound.play()                    # ゲームオーバー音をコメントアウト
            self.hit = 0
            self.score.set_score(0)                         # スコアを0点にする
            #self.score.add_score(-100)                      # スコア減点-100点

        # ボールと衝突したブロックリストを取得（Groupが格納しているSprite中から、指定したSpriteと接触しているものを探索）
        blocks_collided = pygame.sprite.spritecollide(self, self.blocks, True)
        if blocks_collided:  # 衝突ブロックがある場合
            oldrect = self.rect
            for block in blocks_collided:
                # ボールが左からブロックへ衝突した場合
                if oldrect.left < block.rect.left and oldrect.right < block.rect.right:
                    self.rect.right = block.rect.left
                    self.dx = -self.dx
                    
                # ボールが右からブロックへ衝突した場合
                if block.rect.left < oldrect.left and block.rect.right < oldrect.right:
                    self.rect.left = block.rect.right
                    self.dx = -self.dx

                # ボールが上からブロックへ衝突した場合
                if oldrect.top < block.rect.top and oldrect.bottom < block.rect.bottom:
                    self.rect.bottom = block.rect.top
                    self.dy = -self.dy

                # ボールが下からブロックへ衝突した場合
                if block.rect.top < oldrect.top and block.rect.bottom < oldrect.bottom:
                    self.rect.top = block.rect.bottom
                    self.dy = -self.dy
                # self.block_sound.play()    # 効果音を鳴らす処理をコメントアウト
                self.hit += 1              # 衝突回数をカウント
                self.score.add_score(self.hit * 10)   # 衝突回数に応じてスコア加点

# ブロック
class Block(pygame.sprite.Sprite):
    def __init__(self, filename, x, y):
        pygame.sprite.Sprite.__init__(self, self.containers)
        self.image = pygame.image.load(filename).convert()
        self.rect = self.image.get_rect()
        # ブロックの左上座標
        self.rect.left = SCREEN.left + x * self.rect.width
        self.rect.top = SCREEN.top + y * self.rect.height

# スコア
class Score():
    def __init__(self, x, y):
        self.sysfont = pygame.font.SysFont(None, 20)
        self.score = 0
        (self.x, self.y) = (x, y)
    def draw(self, screen):
        img = self.sysfont.render("SCORE:" + str(self.score), True, (255,255,250))
        screen.blit(img, (self.x, self.y))
    def add_score(self, x):
        self.score += x
    def set_score(self, score):
        self.score = score

def main():

    pygame.init()
    screen = pygame.display.set_mode(SCREEN.size)
    # 以下のサウンド読み込み処理は不要なので削除またはコメントアウトのままにします
    #Ball.paddle_sound = pygame.mixer.Sound(PADDLE_SOUND_PATH)    # パドルにボールが衝突した時の効果音取得
    #Ball.block_sound = pygame.mixer.Sound(BLOCK_SOUND_PATH)    # ブロックにボールが衝突した時の効果音取得
    #Ball.gameover_sound = pygame.mixer.Sound(GAMEOVER_SOUND_PATH)    # ゲームオーバー時の効果音取得
    
    # 描画用のスプライトグループ
    group = pygame.sprite.RenderUpdates()  

    # 衝突判定用のスプライトグループ
    blocks = pygame.sprite.Group()   

    # スプライトグループに追加    
    Paddle.containers = group
    Ball.containers = group
    Block.containers = group, blocks

    # パドルの作成
    paddle = Paddle(PADDLE_IMG_PATH)

    # ブロックの作成(14*10)
    for x in range(1, 15):
        for y in range(1, 11):
            Block(BLOCK_IMG_PATH, x, y)

    # スコアを画面(10, 10)に表示
    score = Score(10, 10)    

    # ボールを作成
    Ball(BALL_IMG_PATH, paddle, blocks, score, 5, 135, 45)
    
    clock = pygame.time.Clock()

    running = True  # ループ処理の実行を継続するフラグ

    while running:
        clock.tick(60)      # フレームレート(60fps)
        screen.fill((0,20,0))
        # 全てのスプライトグループを更新
        group.update()
        # 全てのスプライトグループを描画      
        group.draw(screen)
        # スコアを描画   
        score.draw(screen) 
        # 画面更新 
        pygame.display.update()

        # イベント処理
        for event in pygame.event.get():
            # 閉じるボタンが押されたら終了
            if event.type == QUIT: 
                running = False
            # キーイベント
            if event.type == KEYDOWN:
                # Escキーが押されたら終了
                if event.key == K_ESCAPE:   
                    running = False
    # 終了処理
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

In [ ]:
#test2.py
# -*- coding: utf-8 -*-
import math
import sys
import pygame
from pygame.locals import *

# 画面サイズ
SCREEN = Rect(0, 0, 500, 600)

# 画像ファイルのパス
PADDLE_IMG_PATH = "image/paddle.png"
BLOCK_IMG_PATH = "image/backsnow.png"
BLOCKSTOP_IMG_PATH = "image/blockcatch.png"
BALL_IMG_PATH = "image/ball.png"

# バドルのスプライトクラス
class Paddle(pygame.sprite.Sprite):
    # コンストラクタ（初期化メソッド）
    def __init__(self, filename):
        pygame.sprite.Sprite.__init__(self, self.containers)
        self.image = pygame.image.load(filename).convert()
        self.rect = self.image.get_rect()
        self.rect.bottom = SCREEN.bottom - 20     # パドルのy座標

    def update(self):
        self.rect.centerx = pygame.mouse.get_pos()[0]  # マウスのx座標をパドルのx座標に
        self.rect.clamp_ip(SCREEN)                      # ゲーム画面内のみで移動

# ボールのスプライトクラス
class Ball(pygame.sprite.Sprite):
    # コンストラクタ（初期化メソッド）
    def __init__(self, filename, paddle, blocks, score, speed, angle_left, angle_right):
        pygame.sprite.Sprite.__init__(self, self.containers)
        self.image = pygame.image.load(filename).convert()
        self.rect = self.image.get_rect()
        self.dx = self.dy = 0  # ボールの速度
        self.paddle = paddle  # パドルへの参照
        self.blocks = blocks  # ブロックグループへの参照
        self.update = self.start # ゲーム開始状態に更新
        self.score = score
        self.hit = 0  # 連続でブロックを壊した回数
        self.speed = speed # ボールの初期速度
        self.angle_left = angle_left # パドルの反射方向(左端:135度）
        self.angle_right = angle_right # パドルの反射方向(右端:45度）
        self.is_game_over = False # ▼▼▼ Game Overフラグを追加 ▼▼▼

    # ゲーム開始状態（マウスを左クリック時するとボール射出）
    def start(self):
        self.rect.centerx = self.paddle.rect.centerx
        self.rect.bottom = self.paddle.rect.top
        if pygame.mouse.get_pressed()[0] == 1:
            self.dx = 0
            self.dy = -self.speed
            self.update = self.move

    # ボールの挙動
    def move(self):
        self.rect.centerx += self.dx
        self.rect.centery += self.dy
        if self.rect.left < SCREEN.left:
            self.rect.left = SCREEN.left
            self.dx = -self.dx
        if self.rect.right > SCREEN.right:
            self.rect.right = SCREEN.right
            self.dx = -self.dx
        if self.rect.top < SCREEN.top:
            self.rect.top = SCREEN.top
            self.dy = -self.dy
        if self.rect.colliderect(self.paddle.rect) and self.dy > 0:
            self.hit = 0
            (x1, y1) = (self.paddle.rect.left - self.rect.width, self.angle_left)
            (x2, y2) = (self.paddle.rect.right, self.angle_right)
            x = self.rect.left
            y = (float(y2-y1)/(x2-x1)) * (x - x1) + y1
            angle = math.radians(y)
            self.dx = self.speed * math.cos(angle)
            self.dy = -self.speed * math.sin(angle)

        # ▼▼▼ ボールを落としたら is_game_over フラグを True にする ▼▼▼
        if self.rect.top > SCREEN.bottom:
            self.is_game_over = True

        blocks_collided = pygame.sprite.spritecollide(self, self.blocks, True)
        if blocks_collided:
            oldrect = self.rect
            for block in blocks_collided:
                if oldrect.left < block.rect.left and oldrect.right < block.rect.right:
                    self.rect.right = block.rect.left
                    self.dx = -self.dx
                if block.rect.left < oldrect.left and block.rect.right < oldrect.right:
                    self.rect.left = block.rect.right
                    self.dx = -self.dx
                if oldrect.top < block.rect.top and oldrect.bottom < block.rect.bottom:
                    self.rect.bottom = block.rect.top
                    self.dy = -self.dy
                if block.rect.top < oldrect.top and block.rect.bottom < oldrect.bottom:
                    self.rect.top = block.rect.bottom
                    self.dy = -self.dy
                self.hit += 1
                self.score.add_score(self.hit * 10)

# ブロック
class Block(pygame.sprite.Sprite):
    def __init__(self, filename, x, y):
        pygame.sprite.Sprite.__init__(self, self.containers)
        self.image = pygame.image.load(filename).convert()
        self.rect = self.image.get_rect()
        self.rect.left = SCREEN.left + x * self.rect.width
        self.rect.top = SCREEN.top + y * self.rect.height

# スコア
class Score():
    def __init__(self, x, y):
        self.sysfont = pygame.font.SysFont(None, 20)
        self.score = 0
        (self.x, self.y) = (x, y)
    def draw(self, screen):
        img = self.sysfont.render("SCORE:" + str(self.score), True, (255,255,250))
        screen.blit(img, (self.x, self.y))
    def add_score(self, x):
        self.score += x
    def set_score(self, score):
        self.score = score

def main():
    pygame.init()
    screen = pygame.display.set_mode(SCREEN.size)
    
    # スプライトグループの準備
    group = pygame.sprite.RenderUpdates()
    blocks = pygame.sprite.Group()
    Paddle.containers = group
    Ball.containers = group
    Block.containers = group, blocks

    # --- ゲーム要素の初期作成 ---
    paddle = Paddle(PADDLE_IMG_PATH)
    score = Score(10, 10)

    # ▼▼▼ ブロックの作成(10*10)に変更 ▼▼▼
    # マージンを考慮して配置
    block_width = pygame.image.load(BLOCK_IMG_PATH).get_width()
    block_height = pygame.image.load(BLOCK_IMG_PATH).get_height()
    for x in range(10):
        for y in range(10):
            # 画面サイズとブロック数から配置位置を計算し、中央に寄せる
            px = 50 + x * (block_width + 5)
            py = 50 + y * (block_height + 5)
            Block(BLOCK_IMG_PATH, px / block_width, py / block_height)
    
    ball = Ball(BALL_IMG_PATH, paddle, blocks, score, 7, 135, 45) # ボールのスピードを少し上げた
    
    clock = pygame.time.Clock()

    # --- ゲームオーバーとクリア画面用のUI準備 ---
    game_over_font = pygame.font.SysFont(None, 80)
    clear_font = pygame.font.SysFont(None, 100)
    retry_font = pygame.font.SysFont(None, 50)

    # --- ゲームの状態を管理する変数 ---
    game_state = "playing" # "playing", "game_over", "clear"

    running = True
    while running:
        clock.tick(60)
        screen.fill((0, 20, 0))

        # --- イベント処理 ---
        for event in pygame.event.get():
            if event.type == QUIT:
                running = False
            if event.type == KEYDOWN and event.key == K_ESCAPE:
                running = False
            
            # ▼▼▼ リトライボタンのクリック処理 ▼▼▼
            if event.type == MOUSEBUTTONDOWN and game_state == "game_over":
                # retry_button_rectは後で定義
                if retry_button_rect.collidepoint(event.pos):
                    # --- ゲームのリセット処理 ---
                    game_state = "playing"
                    # 全てのスプライトを削除
                    group.empty()
                    blocks.empty()
                    # スプライトを再作成
                    paddle = Paddle(PADDLE_IMG_PATH)
                    score.set_score(0)
                    for x in range(10):
                        for y in range(10):
                            px = 50 + x * (block_width + 5)
                            py = 50 + y * (block_height + 5)
                            Block(BLOCK_IMG_PATH, px/block_width, py/block_height)
                    ball = Ball(BALL_IMG_PATH, paddle, blocks, score, 7, 135, 45)

        # --- ゲームの状態に応じた処理 ---
        if game_state == "playing":
            group.update()
            group.draw(screen)
            score.draw(screen)

            # ボールが落ちたらゲームオーバーに移行
            if ball.is_game_over:
                game_state = "game_over"
            
            # ブロックがなくなったらクリアに移行
            if len(blocks) == 0:
                game_state = "clear"

        elif game_state == "game_over":
            # --- ゲームオーバー画面の描画 ---
            # "Gameover"の文字
            game_over_text = game_over_font.render("Game Over", True, (255, 0, 0))
            text_rect = game_over_text.get_rect(center=(SCREEN.centerx, SCREEN.centery - 50))
            screen.blit(game_over_text, text_rect)
            
            # スコア表示
            final_score_text = score.sysfont.render("SCORE: " + str(score.score), True, (255, 255, 255))
            score_rect = final_score_text.get_rect(center=(SCREEN.centerx, SCREEN.centery + 20))
            screen.blit(final_score_text, score_rect)

            # リトライボタン
            retry_text = retry_font.render("Retry", True, (0, 0, 0))
            retry_button_rect = pygame.Rect(SCREEN.centerx - 70, SCREEN.centery + 60, 140, 50)
            pygame.draw.rect(screen, (255, 255, 255), retry_button_rect)
            retry_text_rect = retry_text.get_rect(center=retry_button_rect.center)
            screen.blit(retry_text, retry_text_rect)

        elif game_state == "clear":
            # --- クリア画面の描画 ---
            clear_text = clear_font.render("Nice try", True, (255, 255, 0))
            text_rect = clear_text.get_rect(center=SCREEN.center)
            screen.blit(clear_text, text_rect)
        
        pygame.display.update()

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

In [ ]:
#test3.py
# -*- coding: utf-8 -*-
import math
import sys
import pygame
from pygame.locals import *
import random

# 画面サイズ
SCREEN = Rect(0, 0, 450, 600)

# 画像ファイルのパス
PADDLE_IMG_PATH = "image/paddle.png"
BLOCK_IMG_PATH = "image/backsnow.png"      # 壊せるブロック (体力1)
BLOCKSTOP_IMG_PATH = "image/blockcatch.png" # 硬いブロック (体力2)
BALL_IMG_PATH = "image/ball.png"

# バドルのスプライトクラス
class Paddle(pygame.sprite.Sprite):
    def __init__(self, filename, *groups):
        pygame.sprite.Sprite.__init__(self, *groups)
        self.image = pygame.image.load(filename).convert()
        self.rect = self.image.get_rect()
        self.rect.bottom = SCREEN.bottom - 20

    def update(self):
        self.rect.centerx = pygame.mouse.get_pos()[0]
        self.rect.clamp_ip(SCREEN)

# ボールのスプライトクラス
class Ball(pygame.sprite.Sprite):
    # ▼▼▼ blocksグループを1つに統合 ▼▼▼
    def __init__(self, filename, paddle, blocks, score, speed, angle_left, angle_right, *groups):
        pygame.sprite.Sprite.__init__(self, *groups)
        self.image = pygame.image.load(filename).convert()
        self.rect = self.image.get_rect()
        self.dx = self.dy = 0
        self.paddle = paddle
        self.blocks = blocks # すべてのブロックが入ったグループ
        self.update = self.start
        self.score = score
        self.hit = 0
        self.speed = speed
        self.angle_left = angle_left
        self.angle_right = angle_right
        self.is_game_over = False

    def start(self):
        self.rect.centerx = self.paddle.rect.centerx
        self.rect.bottom = self.paddle.rect.top
        if pygame.mouse.get_pressed()[0] == 1:
            self.dx = 0
            self.dy = -self.speed
            self.update = self.move

    def move(self):
        self.rect.centerx += self.dx
        self.rect.centery += self.dy
        if self.rect.left < SCREEN.left:
            self.rect.left = SCREEN.left
            self.dx = -self.dx
        if self.rect.right > SCREEN.right:
            self.rect.right = SCREEN.right
            self.dx = -self.dx
        if self.rect.top < SCREEN.top:
            self.rect.top = SCREEN.top
            self.dy = -self.dy
        if self.rect.colliderect(self.paddle.rect) and self.dy > 0:
            self.hit = 0
            (x1, y1) = (self.paddle.rect.left - self.rect.width, self.angle_left)
            (x2, y2) = (self.paddle.rect.right, self.angle_right)
            x = self.rect.left
            y = (float(y2-y1)/(x2-x1)) * (x - x1) + y1
            angle = math.radians(y)
            self.dx = self.speed * math.cos(angle)
            self.dy = -self.speed * math.sin(angle)

        if self.rect.top > SCREEN.bottom:
            self.is_game_over = True

        # ▼▼▼▼▼ 衝突判定ロジックを体力制に変更 ▼▼▼▼▼
        # 衝突したブロックのリストを取得（ブロックはまだ消さない）
        collided_blocks = pygame.sprite.spritecollide(self, self.blocks, False)
        
        if collided_blocks:
            # 最初に衝突したブロックを取得
            block = collided_blocks[0]
            
            # ブロックのhitメソッドを呼び出し、破壊されたかどうかを受け取る
            destroyed = block.hit()
            
            # ブロックが破壊された場合のみスコアを加算
            if destroyed:
                self.hit += 1
                self.score.add_score(self.hit * 10)

            # ボールの反射処理
            oldrect = self.rect
            if oldrect.left < block.rect.left and oldrect.right < block.rect.right:
                self.rect.right = block.rect.left
                self.dx = -self.dx
            if block.rect.left < oldrect.left and block.rect.right < oldrect.right:
                self.rect.left = block.rect.right
                self.dx = -self.dx
            if oldrect.top < block.rect.top and oldrect.bottom < block.rect.bottom:
                self.rect.bottom = block.rect.top
                self.dy = -self.dy
            if block.rect.top < oldrect.top and block.rect.bottom < oldrect.bottom:
                self.rect.top = block.rect.bottom
                self.dy = -self.dy
        # ▲▲▲▲▲ 衝突判定ロジックの変更ここまで ▲▲▲▲▲

# ブロック
class Block(pygame.sprite.Sprite):
    # ▼▼▼ 体力(health)をコンストラクタで受け取るように変更 ▼▼▼
    def __init__(self, filename, x, y, health, *groups):
        pygame.sprite.Sprite.__init__(self, *groups)
        self.image = pygame.image.load(filename).convert()
        self.rect = self.image.get_rect()
        self.rect.left = x
        self.rect.top = y
        self.health = health # 体力を設定

    # ▼▼▼ ボールが当たった時の処理を追加 ▼▼▼
    def hit(self):
        """ブロックの体力を1減らし、体力が0になったら自身を消去する"""
        self.health -= 1
        if self.health <= 0:
            self.kill() # スプライトグループから自身を削除
            return True # 破壊されたことを通知
        return False # まだ破壊されていないことを通知

# スコア
class Score():
    def __init__(self, x, y):
        self.sysfont = pygame.font.SysFont(None, 20)
        self.score = 0
        (self.x, self.y) = (x, y)
    def draw(self, screen):
        img = self.sysfont.render("SCORE:" + str(self.score), True, (255,255,250))
        screen.blit(img, (self.x, self.y))
    def add_score(self, x):
        self.score += x
    def set_score(self, score):
        self.score = score

def main():
    pygame.init()
    screen = pygame.display.set_mode(SCREEN.size)
    
    # --- スプライトグループの準備 ---
    all_sprites = pygame.sprite.RenderUpdates()
    all_blocks = pygame.sprite.Group()  # ▼▼▼ すべてのブロックを管理するグループ ▼▼▼

    # --- ゲーム要素の初期作成関数 ---
    def setup_game():
        all_sprites.empty()
        all_blocks.empty()
        
        paddle = Paddle(PADDLE_IMG_PATH, all_sprites)
        
        block_width = pygame.image.load(BLOCK_IMG_PATH).get_width()
        block_height = pygame.image.load(BLOCK_IMG_PATH).get_height()
        for y in range(10):
            for x in range(10):
                px = 25 + x * (block_width + 5)
                py = 50 + y * (block_height + 5)
                
                # ▼▼▼ 確率に応じて体力1か体力2のブロックを生成 ▼▼▼
                if random.random() < 0.3: 
                    # 体力2のブロック
                    Block(BLOCKSTOP_IMG_PATH, px, py, 2, all_sprites, all_blocks)
                else:
                    # 体力1のブロック
                    Block(BLOCK_IMG_PATH, px, py, 1, all_sprites, all_blocks)
        
        score = Score(10, 10)
        # ▼▼▼ Ballにはall_blocksグループを渡す ▼▼▼
        ball = Ball(BALL_IMG_PATH, paddle, all_blocks, score, 7, 135, 45, all_sprites)
        
        return paddle, score, ball

    paddle, score, ball = setup_game()
    clock = pygame.time.Clock()

    game_over_font = pygame.font.SysFont(None, 80)
    clear_font = pygame.font.SysFont(None, 100)
    retry_font = pygame.font.SysFont(None, 50)

    game_state = "playing"
    running = True
    while running:
        clock.tick(60)
        screen.fill((0, 20, 0))

        for event in pygame.event.get():
            if event.type == QUIT:
                running = False
            if event.type == KEYDOWN and event.key == K_ESCAPE:
                running = False
            
            if event.type == MOUSEBUTTONDOWN and game_state == "game_over":
                if retry_button_rect.collidepoint(event.pos):
                    paddle, score, ball = setup_game()
                    game_state = "playing"

        if game_state == "playing":
            all_sprites.update()
            all_sprites.draw(screen)
            score.draw(screen)

            if ball.is_game_over:
                game_state = "game_over"
            
            # ▼▼▼ クリア条件を「すべてのブロックが0」に変更 ▼▼▼
            if len(all_blocks) == 0:
                game_state = "clear"

        elif game_state == "game_over":
            game_over_text = game_over_font.render("Game Over", True, (255, 0, 0))
            text_rect = game_over_text.get_rect(center=(SCREEN.centerx, SCREEN.centery - 50))
            screen.blit(game_over_text, text_rect)
            
            final_score_text = score.sysfont.render("SCORE: " + str(score.score), True, (255, 255, 255))
            score_rect = final_score_text.get_rect(center=(SCREEN.centerx, SCREEN.centery + 20))
            screen.blit(final_score_text, score_rect)

            retry_text = retry_font.render("Retry", True, (0, 0, 0))
            retry_button_rect = pygame.Rect(SCREEN.centerx - 70, SCREEN.centery + 60, 140, 50)
            pygame.draw.rect(screen, (255, 255, 255), retry_button_rect)
            retry_text_rect = retry_text.get_rect(center=retry_button_rect.center)
            screen.blit(retry_text, retry_text_rect)

        elif game_state == "clear":
            clear_text = clear_font.render("Nice try", True, (255, 255, 0))
            text_rect = clear_text.get_rect(center=SCREEN.center)
            screen.blit(clear_text, text_rect)
        
        pygame.display.update()

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()



In [ ]:
import mediapipe as mp
import cv2
import random
import math
import time

In [ ]:
#handscdetect.py
#専用ターミナルでpythonファイルを実行
#Qiitaの練習用コード
#・ゲームの説明
#目的: 右手の人差し指で赤い丸に触れてスコアを稼ぐ。
#制限時間: 30秒。
#操作:
#'f' キーでフルスクリーンとウィンドウモードを切り替え。
#'q' キーでゲームを終了。（30秒たつと自動的に終了はします。）

# MediaPipe Handsのセットアップ
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# スコアの初期化
score = 0

# ランダムに赤い丸を表示するための初期位置
circle_x = random.randint(100, 500)
circle_y = random.randint(100, 400)
circle_radius = 20

# カメラからの映像をキャプチャ
cap = cv2.VideoCapture(0)

# OpenCVのウィンドウを作成
cv2.namedWindow('Hand Game', cv2.WINDOW_NORMAL)
is_fullscreen = False

# ゲームの制限時間（秒）
game_duration = 30
start_time = time.time()

def is_hand_touching_circle(hand_x, hand_y, circle_x, circle_y, circle_radius):
    distance = math.sqrt((hand_x - circle_x) ** 2 + (hand_y - circle_y) ** 2)
    return distance < circle_radius

def generate_new_circle_position(frame_width, frame_height, circle_radius, exclude_area):
    while True:
        new_x = random.randint(circle_radius, frame_width - circle_radius)
        new_y = random.randint(circle_radius, frame_height - circle_radius)
        if not (exclude_area[0] <= new_x <= exclude_area[2] and exclude_area[1] <= new_y <= exclude_area[3]):
            return new_x, new_y

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 画像を水平方向に反転
    frame = cv2.flip(frame, 1)

    # 画像をRGBに変換
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 手のランドマークの検出
    results = hands.process(image)

    # 画像をBGRに戻す
    frame = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # スコアとタイマー表示領域の背景を描画
    cv2.rectangle(frame, (0, 0), (200, 100), (0, 0, 0), -1)

    # 検出結果がある場合、ランドマークを描画し、右手の位置を取得
    if results.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # ランドマークの座標を取得（例として8番目のランドマーク=右手の人差し指の先端を使用）
            hand_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * frame.shape[1])
            hand_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * frame.shape[0])

            if is_hand_touching_circle(hand_x, hand_y, circle_x, circle_y, circle_radius):
                score += 1
                # 新しい位置に赤い丸を再配置
                circle_x, circle_y = generate_new_circle_position(frame.shape[1], frame.shape[0], circle_radius, (0, 0, 200, 100))

    # 赤い丸を描画
    cv2.circle(frame, (circle_x, circle_y), circle_radius, (0, 0, 255), -1)

    # スコアを表示
    cv2.putText(frame, f'Score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # 残り時間を計算
    elapsed_time = time.time() - start_time
    remaining_time = max(0, game_duration - int(elapsed_time))
    cv2.putText(frame, f'Time: {remaining_time}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # ゲーム終了条件
    if remaining_time <= 0:
        break

    # 画像を表示
    cv2.imshow('Hand Game', frame)

    # キー入力をチェック
    key = cv2.waitKey(10) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('f'):
        is_fullscreen = not is_fullscreen
        if is_fullscreen:
            cv2.setWindowProperty('Hand Game', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        else:
            cv2.setWindowProperty('Hand Game', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_NORMAL)

# リソースの解放
cap.release()
cv2.destroyAllWindows()

# ゲーム終了メッセージ
print(f'Game Over! Your score is {score}')


In [ ]:
#fase.py
import matplotlib.pyplot as plt
import numpy as np

# 円を描画（顔）
theta = np.linspace(0, 2 * np.pi, 100)
x = np.cos(theta)
y = np.sin(theta)

# 描画
plt.figure(figsize=(5, 5))
plt.plot(x, y, label="Face")  # 顔の輪郭
plt.scatter([0.3, -0.3], [0.5, 0.5], color="black", s=100, label="Eyes")  # 目
plt.plot([0, 0.2], [-0.2, -0.4], color="red", label="Mouth")  # 口
plt.axis("equal")
plt.legend()
plt.show()


In [ ]:
#mp2scroll.py
import cv2
import mediapipe as mp
import pygame
import math

# --- 初期設定 ---

# MediaPipeの手検出モデルと描画ツールを準備
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

# Pygameの初期化
pygame.init()

# Pygameウィンドウの設定
SCREEN_WIDTH, SCREEN_HEIGHT = 1280, 720
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Bouldering Game (Infinite Climb)")

# 色の定義
WHITE = (255, 255, 255)
RED = (255, 0, 0)
SKY_BLUE = (135, 206, 235) # 背景がない場合の空の色

# プレイヤー（カーソル）の設定
cursor_pos = [SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2]
cursor_radius = 15 # 円を小さく
GRAVITY = 5 # 重力（落下速度）

# Webカメラの準備
cap = cv2.VideoCapture(0)

# --- ★無限スクロール用の背景設定 ---
background_tiles = []
bg_image_height = 0
try:
    background_image = pygame.image.load("image/iceclimbrock.png").convert()
    bg_image_height = background_image.get_height()
    
    # 画面を埋めるのに必要なタイル数を計算 (+1はスクロール時の予備)
    num_tiles = math.ceil(SCREEN_HEIGHT / bg_image_height) + 1
    
    # 初期タイルをリストに追加
    for i in range(num_tiles):
        # タイルを画面下から上へ順番に配置
        rect = background_image.get_rect(topleft=(0, SCREEN_HEIGHT - bg_image_height * (i + 1)))
        background_tiles.append(rect)

except FileNotFoundError:
    print("エラー: image/iceclimbrock.png が見つかりません。")
    background_image = None

# 掴んでいる状態を管理する変数
is_holding = False
last_hand_y = 0  # 1フレーム前の手のY座標

# --- 関数定義 ---

def is_hand_open(hand_landmarks):
    """
    手のランドマークから、その手が開いている（パー）か閉じている（グー）かを判定する関数。
    """
    tip_ids = [
        mp_hands.HandLandmark.INDEX_FINGER_TIP, mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
        mp_hands.HandLandmark.RING_FINGER_TIP, mp_hands.HandLandmark.PINKY_TIP
    ]
    pip_ids = [
        mp_hands.HandLandmark.INDEX_FINGER_PIP, mp_hands.HandLandmark.MIDDLE_FINGER_PIP,
        mp_hands.HandLandmark.RING_FINGER_PIP, mp_hands.HandLandmark.PINKY_PIP
    ]
    open_fingers = sum(1 for tip_id, pip_id in zip(tip_ids, pip_ids) 
                       if hand_landmarks.landmark[tip_id].y < hand_landmarks.landmark[pip_id].y)
    return open_fingers >= 3

# --- メインループ ---
running = True
clock = pygame.time.Clock()

while running and cap.isOpened():
    # 1. イベント処理 (PygameとOpenCV)
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_RETURN):
            running = False

    success, image = cap.read()
    if not success: continue

    # 2. 手の検出
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)

    # 3. カメラ映像の表示
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    cv2.imshow('MediaPipe Hands', image)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or cv2.getWindowProperty('MediaPipe Hands', cv2.WND_PROP_VISIBLE) < 1:
        running = False

    # 4. ジェスチャーとゲームロジック
    is_grabbing = False
    active_hand_pos = None

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            is_open = is_hand_open(hand_landmarks)
            mcp_landmark = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
            hand_x = int(mcp_landmark.x * SCREEN_WIDTH)
            hand_y = int(mcp_landmark.y * SCREEN_HEIGHT)
            
            active_hand_pos = (hand_x, hand_y)
            if not is_open:
                is_grabbing = True
                # 複数検出時はどちらかの手で掴んでいればOK
                break

    if active_hand_pos:
        cursor_pos[0], cursor_pos[1] = active_hand_pos
    
    # ★掴みとスクロールのロジック
    drag_amount = 0
    if is_grabbing and active_hand_pos:
        if not is_holding: # 掴んだ瞬間
            is_holding = True
            last_hand_y = active_hand_pos[1]
        
        # 掴んでいる間の移動量を計算
        drag_amount = active_hand_pos[1] - last_hand_y
        last_hand_y = active_hand_pos[1]
    else: # 手を離した
        is_holding = False
        drag_amount = -GRAVITY # 重力で落下（背景が上にスクロール）

    # 背景タイルの位置を更新
    if background_image:
        for tile_rect in background_tiles:
            tile_rect.y += drag_amount

        # ★無限スクロールの管理
        # 一番上のタイルが画面内に見えてきたら、さらにその上にもう一枚追加
        top_tile = background_tiles[0]
        if top_tile.y > -bg_image_height:
             new_rect = background_image.get_rect(topleft=(0, top_tile.y - bg_image_height))
             background_tiles.insert(0, new_rect)

        # 一番下のタイルが完全に画面外に出たら、リストから削除
        bottom_tile = background_tiles[-1]
        if bottom_tile.y > SCREEN_HEIGHT:
            background_tiles.pop()


    # 5. Pygameの描画処理
    if background_image:
        for tile_rect in background_tiles:
            screen.blit(background_image, tile_rect)
    else:
        screen.fill(SKY_BLUE) # 背景がない場合は空の色
    
    # プレイヤー（カーソル）を描画
    pygame.draw.circle(screen, RED, cursor_pos, cursor_radius)
    pygame.display.flip()

    clock.tick(60)

# --- 終了処理 ---
cap.release()
cv2.destroyAllWindows()
pygame.quit()


In [ ]:
#mp3max100.py
import cv2
import mediapipe as mp
import pygame
import math

# --- 初期設定 ---

# MediaPipeの手検出モデルと描画ツールを準備
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

# Pygameの初期化
pygame.init()

# --- ★ゲーム設定と物理定義 ---
PIXELS_PER_METER = 360  # 1メートルあたりのピクセル数 (720px / 2m)
TOTAL_CLIMB_METERS = 100.0
MAX_PULL_METERS = 1.0

TOTAL_CLIMB_PIXELS = int(TOTAL_CLIMB_METERS * PIXELS_PER_METER)
MAX_PULL_PIXELS = int(MAX_PULL_METERS * PIXELS_PER_METER)
GRAVITY = 5 # 落下速度

# Pygameウィンドウの設定
SCREEN_WIDTH, SCREEN_HEIGHT = 1280, 720
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption(f"Bouldering Game ({int(TOTAL_CLIMB_METERS)}m Climb)")

# 色とフォントの定義
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLACK = (0, 0, 0)
SKY_BLUE = (135, 206, 235)
font = pygame.font.Font(None, 50)

# プレイヤー（カーソル）の設定
cursor_pos = [SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2]
cursor_radius = 15

# Webカメラの準備
cap = cv2.VideoCapture(0)

# --- ★100mの壁を生成 ---
full_background = None
try:
    tile_image = pygame.image.load("image/backsnow.png").convert()
    tile_height = tile_image.get_height()
    
    # 100m分の高さを持つ巨大なサーフェスを作成
    full_background = pygame.Surface((SCREEN_WIDTH, TOTAL_CLIMB_PIXELS))
    
    # タイル画像で巨大サーフェスを埋める
    for y in range(0, TOTAL_CLIMB_PIXELS, tile_height):
        full_background.blit(tile_image, (0, y))

except FileNotFoundError:
    print("エラー: image/backsnow.png が見つかりません。")

# 背景スクロール用の変数
if full_background:
    max_scroll = full_background.get_height() - SCREEN_HEIGHT
    world_y_offset = max_scroll # スタート時は一番下
else:
    max_scroll = 0
    world_y_offset = 0

# 掴んでいる状態を管理する変数
is_holding = False
hold_start_y = 0      # 掴み始めた時の「手のY座標」
world_hold_start_y = 0  # 掴み始めた時の「背景のY座標」

# --- 関数定義 ---
def is_hand_open(hand_landmarks):
    tip_ids = [mp_hands.HandLandmark.INDEX_FINGER_TIP, mp_hands.HandLandmark.MIDDLE_FINGER_TIP, mp_hands.HandLandmark.RING_FINGER_TIP, mp_hands.HandLandmark.PINKY_TIP]
    pip_ids = [mp_hands.HandLandmark.INDEX_FINGER_PIP, mp_hands.HandLandmark.MIDDLE_FINGER_PIP, mp_hands.HandLandmark.RING_FINGER_PIP, mp_hands.HandLandmark.PINKY_PIP]
    open_fingers = sum(1 for tip_id, pip_id in zip(tip_ids, pip_ids) if hand_landmarks.landmark[tip_id].y < hand_landmarks.landmark[pip_id].y)
    return open_fingers >= 3

# --- メインループ ---
running = True
clock = pygame.time.Clock()

while running and cap.isOpened():
    # 1. イベント処理 (PygameとOpenCV)
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_RETURN):
            running = False

    success, image = cap.read()
    if not success: continue

    # 2. 手の検出
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)

    # 3. カメラ映像の表示
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    cv2.imshow('MediaPipe Hands', image)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or cv2.getWindowProperty('MediaPipe Hands', cv2.WND_PROP_VISIBLE) < 1:
        running = False

    # 4. ジェスチャーとゲームロジック
    is_grabbing = False
    active_hand_pos = None
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            if not is_hand_open(hand_landmarks): is_grabbing = True
            mcp_landmark = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
            active_hand_pos = (int(mcp_landmark.x * SCREEN_WIDTH), int(mcp_landmark.y * SCREEN_HEIGHT))
            if is_grabbing: break
    if active_hand_pos: cursor_pos[:] = active_hand_pos
    
    # ★掴みとスクロールのロジック（プル制限付き）
    if is_grabbing and active_hand_pos:
        if not is_holding: # 掴んだ瞬間
            is_holding = True
            hold_start_y = active_hand_pos[1]
            world_hold_start_y = world_y_offset
        
        # 掴んだ位置からの移動距離を計算
        pull_distance = active_hand_pos[1] - hold_start_y
        
        # 移動距離が0未満（上に押している）場合は無視
        if pull_distance < 0:
            pull_distance = 0
            
        # ★1回のプル距離を1m（MAX_PULL_PIXELS）に制限
        if pull_distance > MAX_PULL_PIXELS:
            pull_distance = MAX_PULL_PIXELS
            
        # 背景をスクロール
        world_y_offset = world_hold_start_y - pull_distance

    else: # 手を離した
        is_holding = False
        # 重力で落下
        world_y_offset += GRAVITY
    
    # スクロール範囲の制限 (0が頂上、max_scrollが地上)
    if world_y_offset > max_scroll: world_y_offset = max_scroll
    if world_y_offset < 0: world_y_offset = 0

    # 5. Pygameの描画処理
    if full_background:
        screen.blit(full_background, (0, -world_y_offset))
    else:
        screen.fill(SKY_BLUE)
    
    # ★高度表示UIを描画
    height_climbed = (max_scroll - world_y_offset) / PIXELS_PER_METER
    height_text = font.render(f"Height: {height_climbed:.1f} m", True, BLACK)
    # 文字の背景を描画
    pygame.draw.rect(screen, WHITE, (5, 5, height_text.get_width() + 10, height_text.get_height()))
    screen.blit(height_text, (10, 5))
    
    # プレイヤー（カーソル）を描画
    pygame.draw.circle(screen, RED, cursor_pos, cursor_radius)
    pygame.display.flip()

    clock.tick(60)

# --- 終了処理 ---
cap.release()
cv2.destroyAllWindows()
pygame.quit()


In [ ]:
#mediapygamepr.py
import cv2
import mediapipe as mp
import pygame

# --- 初期設定 ---

# MediaPipeの手検出モデルを準備
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=2,                # 最大検出数を2に設定
    min_detection_confidence=0.7,   # 検出信頼度の閾値
    min_tracking_confidence=0.7     # 追跡信頼度の閾値
)

# Pygameの初期化
pygame.init()

# Pygameウィンドウの設定
SCREEN_WIDTH, SCREEN_HEIGHT = 1280, 720
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Bouldering Game Prototype")

# 色の定義
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)

# プレイヤー（円）の設定
player_pos = [SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2]
player_radius = 30
GRAVITY = 10

# Webカメラの準備
cap = cv2.VideoCapture(0)

# --- 関数定義 ---

def is_hand_open(hand_landmarks):
    """
    手のランドマークから、その手が開いている（パー）か閉じている（グー）かを判定する関数。
    4本の指先が、それぞれの第二関節より上にあるかどうかで判定する。
    """
    # 親指を除く4本の指の先端のランドマークID
    tip_ids = [
        mp_hands.HandLandmark.INDEX_FINGER_TIP,
        mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
        mp_hands.HandLandmark.RING_FINGER_TIP,
        mp_hands.HandLandmark.PINKY_TIP
    ]
    # 親指を除く4本の指の第二関節のランドマークID
    pip_ids = [
        mp_hands.HandLandmark.INDEX_FINGER_PIP,
        mp_hands.HandLandmark.MIDDLE_FINGER_PIP,
        mp_hands.HandLandmark.RING_FINGER_PIP,
        mp_hands.HandLandmark.PINKY_PIP
    ]

    open_fingers = 0
    for tip_id, pip_id in zip(tip_ids, pip_ids):
        # 指先のy座標が第二関節のy座標より小さい（画面上で上にある）場合、指は伸びていると判定
        if hand_landmarks.landmark[tip_id].y < hand_landmarks.landmark[pip_id].y:
            open_fingers += 1

    # 3本以上の指が伸びていれば「パー」と判定
    return open_fingers >= 3


# --- メインループ ---
running = True
clock = pygame.time.Clock()

while running and cap.isOpened():
    # 1. Pygameのイベント処理
    # 1. Pygameのイベント処理
    for event in pygame.event.get():
        # ウィンドウの閉じるボタンが押された場合
        if event.type == pygame.QUIT:
            running = False
        # キーが押された場合
        if event.type == pygame.KEYDOWN:
            # そのキーがエンターキーの場合
            if event.key == pygame.K_RETURN:
                running = False

    # 2. OpenCVとMediaPipeによる手の検出
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # 映像を左右反転させてから、色をBGRからRGBに変換
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    
    # パフォーマンス向上のため、画像を書き込み不可にする
    image.flags.writeable = False
    results = hands.process(image)

    # 3. ジェスチャーと手の位置を判断
    left_hand_open, right_hand_open = False, False
    left_hand_pos, right_hand_pos = None, None

    if results.multi_hand_landmarks:
        # 検出された各手についてループ
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            # ランドマークから手が「パー」か「グー」かを判定
            is_open = is_hand_open(hand_landmarks)
            
            # ランドマークから中指の付け根（第3関節）の座標を取得
            mcp_landmark = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
            hand_x = int(mcp_landmark.x * SCREEN_WIDTH)
            hand_y = int(mcp_landmark.y * SCREEN_HEIGHT)

            # 左右の手の情報を更新
            if handedness.classification[0].label == 'Left':
                left_hand_open = is_open
                left_hand_pos = (hand_x, hand_y)
            elif handedness.classification[0].label == 'Right':
                right_hand_open = is_open
                right_hand_pos = (hand_x, hand_y)

    # 4. ゲームロジック（ジェスチャーに応じてプレイヤーを動かす）
    # 両手グー：止まる
    if not left_hand_open and not right_hand_open:
        pass # 位置を更新しない
    # 右手グー、左手パー：左手の位置に追従
    elif not right_hand_open and left_hand_open and left_hand_pos:
        player_pos[0], player_pos[1] = left_hand_pos
    # 左手グー、右手パー：右手の位置に追従
    elif not left_hand_open and right_hand_open and right_hand_pos:
        player_pos[0], player_pos[1] = right_hand_pos
    # 両手パー：下に落下
    elif left_hand_open and right_hand_open:
        player_pos[1] += GRAVITY
    
    # 画面外に出ないように制限
    if player_pos[1] > SCREEN_HEIGHT - player_radius:
        player_pos[1] = SCREEN_HEIGHT - player_radius


    # 5. Pygameの描画処理
    screen.fill(WHITE) # 画面を黒で塗りつぶす
    pygame.draw.circle(screen, RED, player_pos, player_radius) # プレイヤー（円）を描画

    # 画面を更新
    pygame.display.flip()

    # フレームレートを60に設定
    clock.tick(60)

# --- 終了処理 ---
cap.release()
pygame.quit()
